Setup

In [1]:
import time
import argparse
import json
import csv

from qiskit import QuantumCircuit
from qiskit.transpiler import CouplingMap, PassManager
from pathlib import Path
from typing import Dict, List, Iterable
from util import EAGLE_COUPLING, sabre, count_swaps 
from multilevel_sabre import MultiLevelSabre

Coupling map

In [2]:
from util import EAGLE_COUPLING

coupling_map = CouplingMap(couplinglist=EAGLE_COUPLING)
coupling_map.make_symmetric()
#coupling_map.draw()

Select circuits

In [3]:
directory = "circuits/"
circuit_files = []
for file in Path(directory).glob("*.qasm"):
    circuit_files.append(file)
print(f"Found {len(circuit_files)} circuit files.")    

Found 12 circuit files.


Run circuits through Sabre and Multi-Level Sabre

In [4]:
def run_comparison_example(qasm_path: Path) -> Dict:
    circuit = QuantumCircuit.from_qasm_file(qasm_path)

    # Run SABRE
    print("\nRunning SABRE...")
    start_time = time.time()
    sabre_swaps, sabre_circuit = sabre(
        circuit=circuit,
        coupling=EAGLE_COUPLING,
        random_seed=1
    )
    sabre_time = time.time() - start_time
    sabre_ops = sabre_circuit.count_ops()

    # Run MultiLevel SABRE
    print("\nRunning MultiLevel SABRE...")
    start_time = time.time()
    multilevel_pass = PassManager([
        MultiLevelSabre(
            coupling_graph=coupling_map,
            cycles=10,
            random_seed=1,
            coarsest_solving_trials=50,
            num_interpolation=10,
            use_initial_embedding=True,
            verbose=0
        )
    ])

    multilevel_circuit = multilevel_pass.run(circuit)
    multilevel_time = time.time() - start_time
    multilevel_ops = multilevel_circuit.count_ops()
    multilevel_swaps = count_swaps(multilevel_circuit)

    # Prepare metrics row (keep ops as JSON strings for CSV-friendliness)
    row = {
        "qasm_file": str(qasm_path),
        "sabre_swaps": sabre_swaps,
        "sabre_time_s": round(sabre_time, 4),
        "sabre_depth": sabre_circuit.depth(),
        "sabre_2q_depth": sabre_circuit.depth(lambda x: x.operation.num_qubits == 2),
        "sabre_size": sabre_circuit.size(),
        "sabre_ops_json": json.dumps(sabre_ops, default=int),

        "multilevel_swaps": multilevel_swaps,
        "multilevel_time_s": round(multilevel_time, 4),
        "multilevel_depth": multilevel_circuit.depth(),
        "multilevel_2q_depth": multilevel_circuit.depth(lambda x: x.operation.num_qubits == 2),
        "multilevel_size": multilevel_circuit.size(),
        "multilevel_ops_json": json.dumps(multilevel_ops, default=int),
    }

    # Derived metrics
    row["speedup"] = round(row["sabre_time_s"] / row["multilevel_time_s"], 4) if row["multilevel_time_s"] > 0 else float("inf")
    row["swap_reduction_pct"] = round(((row["sabre_swaps"] - row["multilevel_swaps"]) / row["sabre_swaps"] * 100), 3) if row["sabre_swaps"] else 0.0

    # Pretty print to terminal
    print("\nComparison Results:")
    print("SABRE:")
    print(f"  - Number of SWAPs: {row['sabre_swaps']}")
    print(f"  - Compilation time: {row['sabre_time_s']:.2f} seconds")
    print(f"  - Circuit depth: {row['sabre_depth']}")
    print(f"  - Circuit 2Q depth: {row['sabre_2q_depth']}")
    print(f"  - Circuit size: {row['sabre_size']}")
    print("\nMultiLevel SABRE:")
    print(f"  - Number of SWAPs: {row['multilevel_swaps']}")
    print(f"  - Compilation time: {row['multilevel_time_s']:.2f} seconds")
    print(f"  - Circuit depth: {row['multilevel_depth']}")
    print(f"  - Circuit 2Q depth: {row['multilevel_2q_depth']}")
    print(f"  - Circuit size: {row['multilevel_size']}")
    print(f"\nSpeedup: {row['speedup']:.2f}x")
    print(f"SWAP reduction: {row['swap_reduction_pct']:.1f}%")
    
    return row


In [5]:
rows = []
for qasm_file in circuit_files:
    rows.append(run_comparison_example(qasm_file))



Running SABRE...

Running MultiLevel SABRE...

Comparison Results:
SABRE:
  - Number of SWAPs: 24
  - Compilation time: 0.12 seconds
  - Circuit depth: 259
  - Circuit 2Q depth: 161
  - Circuit size: 611

MultiLevel SABRE:
  - Number of SWAPs: 17
  - Compilation time: 0.77 seconds
  - Circuit depth: 254
  - Circuit 2Q depth: 153
  - Circuit size: 604

Speedup: 0.15x
SWAP reduction: 29.2%

Running SABRE...

Running MultiLevel SABRE...

Comparison Results:
SABRE:
  - Number of SWAPs: 0
  - Compilation time: 0.02 seconds
  - Circuit depth: 44
  - Circuit 2Q depth: 0
  - Circuit size: 126

MultiLevel SABRE:
  - Number of SWAPs: 0
  - Compilation time: 0.03 seconds
  - Circuit depth: 44
  - Circuit 2Q depth: 0
  - Circuit size: 126

Speedup: 0.66x
SWAP reduction: 0.0%

Running SABRE...

Running MultiLevel SABRE...

Comparison Results:
SABRE:
  - Number of SWAPs: 0
  - Compilation time: 0.02 seconds
  - Circuit depth: 36
  - Circuit 2Q depth: 0
  - Circuit size: 102

MultiLevel SABRE:
  - N

Select filepath for results to be saved

In [6]:
def write_csv(rows: List[Dict], out_csv: Path) -> None:
    if not rows:
        print("No rows to write.")
        return
    fieldnames = [
        "qasm_file",
        "sabre_swaps", "sabre_time_s", "sabre_depth", "sabre_2q_depth", "sabre_size", "sabre_ops_json",
        "multilevel_swaps", "multilevel_time_s", "multilevel_depth", "multilevel_2q_depth", "multilevel_size", "multilevel_ops_json",
        "speedup", "swap_reduction_pct",
        "error",
    ]
    out_csv.parent.mkdir(parents=True, exist_ok=True)
    with out_csv.open("w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for r in rows:
            writer.writerow({k: r.get(k, "") for k in fieldnames})
    print(f"\nWrote CSV: {out_csv}")

result_path = Path("results/comparison_test.csv")

# Save results to CSV
write_csv(rows, result_path)


Wrote CSV: results/comparison_test.csv


In [7]:
rows

[{'qasm_file': 'circuits/wstate_n118.qasm',
  'sabre_swaps': 24,
  'sabre_time_s': 0.1172,
  'sabre_depth': 259,
  'sabre_2q_depth': 161,
  'sabre_size': 611,
  'sabre_ops_json': '{"ry": 234, "measure": 118, "cz": 117, "cx": 117, "swap": 24, "x": 1, "barrier": 1}',
  'multilevel_swaps': 17,
  'multilevel_time_s': 0.7724,
  'multilevel_depth': 254,
  'multilevel_2q_depth': 153,
  'multilevel_size': 604,
  'multilevel_ops_json': '{"ry": 234, "measure": 118, "cz": 117, "cx": 117, "swap": 17, "x": 1, "barrier": 1}',
  'speedup': 0.1517,
  'swap_reduction_pct': 29.167},
 {'qasm_file': 'circuits/swap_test_n83.qasm',
  'sabre_swaps': 0,
  'sabre_time_s': 0.0171,
  'sabre_depth': 44,
  'sabre_2q_depth': 0,
  'sabre_size': 126,
  'sabre_ops_json': '{"rx": 82, "cswap": 41, "h": 2, "measure": 1}',
  'multilevel_swaps': 0,
  'multilevel_time_s': 0.026,
  'multilevel_depth': 44,
  'multilevel_2q_depth': 0,
  'multilevel_size': 126,
  'multilevel_ops_json': '{"rx": 82, "cswap": 41, "h": 2, "measure"